In [ ]:
import os

In [ ]:
path_cat = "D:/ashikh/work/CV/PetImages/Cat/"
path_dog = "D:/ashikh/work/CV/PetImages/Dog/"
file_name = '{}.jpg'
new_cat = 'cat.'
new_dog = 'dog.'

In [ ]:
for i in range(12500):
    os.rename(path_dog + file_name.format(str(i)), path_dog + new_dog + file_name.format(str(i)))

In [ ]:
import os
import shutil
import time

In [ ]:
n_name = ['Cat', 'Dog']
choise_name = ['train', 'val', 'test']
path = "D:/ashikh/work/CV/PetImages/"
#for c in choise_name:
#    os.mkdir(path + c)
#    for d in n_name:
#        os.mkdir(path + c + '/' + d)
        

In [ ]:
file_cat = "cat.{}.jpg"
file_dog = "dog.{}.jpg"

In [ ]:
# Генерим тестовую выборку

for i in range(0, 4001):
    if os.path.exists(path_cat + file_cat.format(str(i))):
        shutil.copy(path_cat + file_cat.format(str(i)), path + choise_name[0] + '/' + n_name[0] + '/' + file_cat.format(str(i)))
for i in range(0, 4001):
    if os.path.exists(path_dog + file_dog.format(str(i))):
        shutil.copy(path_dog + file_dog.format(str(i)), path + choise_name[0] + '/' + n_name[1] + '/' + file_dog.format(str(i)))

In [ ]:
# Генерим проверочную выборку

for i in range(4001, 8002):
    if os.path.exists(path_cat + file_cat.format(str(i))):
        shutil.copy(path_cat + file_cat.format(str(i)), path + choise_name[1] + '/' + n_name[0] + '/' + file_cat.format(str(i)))
for i in range(4001, 8002):
    if os.path.exists(path_dog + file_dog.format(str(i))):
        shutil.copy(path_dog + file_dog.format(str(i)), path + choise_name[1] + '/' + n_name[1] + '/' + file_dog.format(str(i)))

In [ ]:
%%time
# Генерим тестовую выборку

for i in range(8002, 12003):
    if os.path.exists(path_cat + file_cat.format(str(i))):
        shutil.copy(path_cat + file_cat.format(str(i)), path + choise_name[2] + '/' + n_name[0] + '/' + file_cat.format(str(i)))
for i in range(8002, 12003):
    if os.path.exists(path_dog + file_dog.format(str(i))):
        shutil.copy(path_dog + file_dog.format(str(i)), path + choise_name[2] + '/' + n_name[1] + '/' + file_dog.format(str(i)))

In [ ]:
# CV

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
train_dir = 'D:/ashikh/work/CV/AnimalFaces_choice/train'
val_dir = 'D:/ashikh/work/CV/AnimalFaces_choice/test'
test_dir = 'D:/ashikh/work/CV/AnimalFaces_choice/test'
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 30
batch_size = 10
nb_train_samples = 580
nb_validation_samples = 130
nb_test_samples = 130

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale = 1. / 255)

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

In [ ]:
# Обучение 

model.fit_generator(
        train_generator,
        steps_per_epoch = nb_train_samples // batch_size,
        epochs = epochs,
        validation_data = val_generator,
        validation_steps = nb_validation_samples // batch_size)


In [ ]:
model.save("D:/ashikh/work/CV/PetImages/mymodel4")

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print('Accuracy: %.3f%%' % (scores[1]*100))

In [ ]:
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
model = keras.models.load_model("D:/ashikh/work/CV/PetImages/mymodel4")

In [ ]:
img_path = "D:/ashikh/work/CV/pandafrontal0010.jpg"
img = image.load_img(img_path, target_size = (150, 150))

In [ ]:
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
p = model.predict(x)

In [ ]:
p

In [ ]:
print('кошка' if p[0] == 0 else 'собака')

In [ ]:
#Как я могу прервать обучение, когда потеря валидации больше не уменьшается?
#вы можете использовать обратный вызов EarlyStopping:

from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor=’val_loss’, patience=2)

model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])



In [ ]:
# Определитель битых изображений

import glob
import os
from PIL import Image
from skimage.io import imread
from skimage.transform  import resize

imagepath = "D:/ashikh/work/CV/AnimalFaces_choice/train"
for subdir in os.listdir(imagepath):    
    curr_path = os.path.join(imagepath, subdir)
    for imgname in os.listdir(curr_path):
        if imgname[-3:] in {'jpg', 'png'}:
            im = imread(os.path.join(curr_path, imgname))
            im = resize(im, (150, 150))
            if im.shape != (150,150,3):
                print(im.shape)
                print(curr_path, imgname)
            if img is None:
                print(imgname)

In [ ]:
imgname = "dog000592a.jpg"
imgpath = "D:/ashikh/work/CV/AnimalFaces_choice/train/DogHead"
im = imread(os.path.join(imgpath, imgname))
im = resize(im, (150, 150))
print(im.shape)